<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=51e4ca35ed78cad8b5ff5e8ce25a8d0e9781868385b1700a1cd9b543a8b9e162
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 11:45:41
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.75 L (-1.26%)
Current PnL: -22.79 L (-15.33%)
CY Booked + Current PnL: -11.35 L (-7.64%)
-------------------
Total profit:  85.67 K
Total loss:  -23.65 L
-------------------
Total Booked + Current PnL: 15.63 L (12.84%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.36%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.66 L (64.81%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,95.30,39.0,M-SC,2.78,84102.0,-16675.0,16770.0,-1.71,-16.55,19.94,0.09,245.0,-0.99,0.61,10.49,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,10.24,49.0,H-MC,3.84,190270.0,25600.0,17733.0,-1.28,15.55,9.32,26.31,89.0,1.44,1.39,24.77,X40N,NTT,AC
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106477.0,5973.0,23137.0,-2.29,5.94,21.73,28.96,79.0,0.26,0.78,41.93,MH,ATH,METALS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215108.0,4742.0,24049.0,0.47,2.25,11.18,13.68,37.0,0.20,1.57,19.15,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-22.16,43.0,M-LC,2.35,245588.0,-30935.0,141017.0,1.84,-11.19,57.42,39.81,54.0,-0.22,1.80,17.70,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.88,82356.0,-19854.0,113248.0,0.62,-19.42,137.51,91.37,208.0,-0.18,0.60,33.69,XY25,NTT,FINANCE
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215108.0,4742.0,24049.0,0.47,2.25,11.18,13.68,37.0,0.20,1.57,19.15,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
34,ICICIGI,2260.25,-18.72,51.0,H-MC,1.86,139712.0,3579.0,27551.0,0.33,2.63,19.72,22.86,91.0,0.13,1.02,15.57,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-3.65,31.0,H-SC,8.76,114748.0,-10549.0,69480.0,-3.23,-8.42,60.55,47.03,141.0,-0.15,0.84,43.68,MH,ATH,POWER
27,HAPPSTMNDS,1488.71,-20.69,27.0,H-SC,10.54,85656.0,-42509.0,152536.0,-3.20,-33.17,178.08,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
32,HINDZINC,730.22,20.84,53.0,M-LC,2.78,196685.0,-8391.0,120961.0,-3.13,-4.09,61.50,54.89,52.0,-0.07,1.44,19.02,X5K,ATH,METALS
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,41.34,78854.0,-34695.0,74746.0,-3.07,-30.55,94.79,35.27,268.0,-0.46,0.58,93.00,XR,NTT,HOTELS
54,RAJESHEXPO,518.00,1681.57,40.0,L-SC,2.61,48614.0,-88563.0,88657.0,-2.95,-64.56,182.37,0.07,267.0,-1.00,0.36,20.91,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17973.08,-30.28,51.0,H-MC,3.4,201825.0,1005.0,67773.0,-2.4,0.5,33.58,34.25,80.0,0.01,1.48,5.72,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,91.37,48.0,H-SC,10.16,120582.0,-21897.0,41420.0,-0.69,-15.37,34.35,13.70,163.0,-0.53,0.88,42.32,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,95.30,39.0,M-SC,2.78,84102.0,-16675.0,16770.0,-1.71,-16.55,19.94,0.09,245.0,-0.99,0.61,10.49,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.40,219384.0,-50283.0,86613.0,-0.60,-18.65,39.48,13.47,138.0,-0.58,1.60,10.96,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,35.0,H-SC,1.84,141036.0,-34867.0,76893.0,0.07,-19.82,54.52,23.89,149.0,-0.45,1.03,21.80,OX40N,NTT,CERAMICS
66,SIS,528.0,2079.79,40.0,H-SC,4.23,87975.0,-23057.0,46662.0,-0.32,-20.77,53.04,21.26,156.0,-0.49,0.64,18.86,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,56.0,M-MC,11.17,228320.0,3358.0,164299.0,-2.88,1.49,71.96,74.53,192.0,0.02,1.67,34.23,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,10.24,49.0,H-MC,3.84,190270.0,25600.0,17733.0,-1.28,15.55,9.32,26.31,89.0,1.44,1.39,24.77,X40N,NTT,AC
84,VOLTAS,1530.00,-3.88,39.0,H-MC,3.02,201615.0,9873.0,27883.0,-2.06,5.15,13.83,19.69,99.0,0.35,1.47,12.30,XY25,NTT,AC
17,CAMS,5211.76,-5.61,44.0,H-SC,1.76,107215.0,5211.0,38715.0,-1.02,5.11,36.11,43.06,122.0,0.13,0.78,23.10,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,6.83,154518.0,13678.0,30811.0,-2.07,9.71,19.94,31.59,66.0,0.44,1.13,38.77,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106477.0,5973.0,23137.0,-2.29,5.94,21.73,28.96,79.0,0.26,0.78,41.93,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-44.58,32.0,M-SC,1.49,93758.0,2260.0,76497.0,0.09,2.47,81.59,86.07,223.0,0.03,0.69,36.15,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.22,181715.0,1907.0,110501.0,-1.34,1.06,60.81,62.52,88.0,0.02,1.33,35.51,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,125106.0,1770.0,131987.0,-0.55,1.44,105.50,108.45,119.0,0.01,0.91,24.11,AR,ATH,MISC
86,WIPRO,420.00,-16.52,33.0,M-LC,5.64,146952.0,-3993.0,113447.0,-2.06,-2.65,77.20,72.51,53.0,-0.04,1.07,3.22,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-17.64,29.0,H-LC,7.04,207812.0,-43956.0,97173.0,-1.77,-17.46,46.76,21.13,27.0,-0.45,1.52,11.78,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.61,29.0,H-LC,12.96,256452.0,-10913.0,76371.0,-1.13,-4.08,29.78,24.48,48.0,-0.14,1.88,48.65,XY24,BTT,FINANCE
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273954.0,-72002.0,146867.0,-1.45,-20.81,53.61,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.49,31.0,H-LC,8.18,286271.0,-29571.0,142248.0,-0.70,-9.36,49.69,35.67,5.0,-0.21,2.09,3.19,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.66,35.0,H-LC,9.93,219737.0,-22187.0,85456.0,-1.70,-9.17,38.89,26.15,8.0,-0.26,1.61,3.84,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,40.0,H-LC,1.67,229410.0,-16741.0,95939.0,-2.22,-6.80,41.82,32.18,16.0,-0.17,1.68,26.79,X200,ATH,IT
42,ITC,452.0,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.04,42.0,H-LC,2.62,155960.0,-30135.0,77621.0,-1.18,-16.19,49.77,25.51,15.0,-0.39,1.14,14.93,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.07,215108.0,4742.0,24049.0,0.47,2.25,11.18,13.68,37.0,0.20,1.57,19.15,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.53,42.0,H-LC,4.51,272142.0,6716.0,50074.0,-0.89,2.53,18.40,21.40,11.0,0.13,1.99,9.80,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273954.0,-72002.0,146867.0,-1.45,-20.81,53.61,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.23,44.0,H-LC,7.16,224804.0,-9340.0,57954.0,-0.80,-3.99,25.78,20.77,86.0,-0.16,1.64,2.65,X40N,NTT,BREWERIES
83,VBL,671.64,-20.49,31.0,H-LC,8.18,286271.0,-29571.0,142248.0,-0.70,-9.36,49.69,35.67,5.0,-0.21,2.09,3.19,X40N,ATH,FMCG
42,ITC,452.00,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.66,35.0,H-LC,9.93,219737.0,-22187.0,85456.0,-1.70,-9.17,38.89,26.15,8.0,-0.26,1.61,3.84,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.04,42.0,H-LC,2.62,155960.0,-30135.0,77621.0,-1.18,-16.19,49.77,25.51,15.0,-0.39,1.14,14.93,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
19,CIPLA,1795.00,-23.55,35.0,H-LC,4.52,203952.0,-548.0,41953.0,-1.39,-0.27,20.57,20.25,10.0,-0.01,1.49,8.44,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.64,29.0,H-LC,7.04,207812.0,-43956.0,97173.0,-1.77,-17.46,46.76,21.13,27.0,-0.45,1.52,11.78,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215108.0,4742.0,24049.0,0.47,2.25,11.18,13.68,37.0,0.20,1.57,19.15,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273954.0,-72002.0,146867.0,-1.45,-20.81,53.61,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-20.32,41.0,H-LC,8.12,311065.0,-1823.0,173512.0,-1.64,-0.58,55.78,54.87,3.0,-0.01,2.27,6.03,X40,BTT,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.19,196382.0,-3756.0,24646.0,0.37,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
83,VBL,671.64,-20.49,31.0,H-LC,8.18,286271.0,-29571.0,142248.0,-0.70,-9.36,49.69,35.67,5.0,-0.21,2.09,3.19,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,43.0,H-LC,7.97,247968.0,-13651.0,132861.0,-0.67,-5.22,53.58,45.57,7.0,-0.10,1.81,5.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,41.34,78854.0,-34695.0,74746.0,-3.07,-30.55,94.79,35.27,268.0,-0.46,0.58,93.00,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.88,82356.0,-19854.0,113248.0,0.62,-19.42,137.51,91.37,208.0,-0.18,0.60,33.69,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.60,40.0,H-SC,12.51,91485.0,-6495.0,28095.0,-1.25,-6.63,30.71,22.05,152.0,-0.23,0.67,32.98,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106477.0,5973.0,23137.0,-2.29,5.94,21.73,28.96,79.0,0.26,0.78,41.93,MH,ATH,METALS
78,UJJIVANSFB,60.00,91.37,48.0,H-SC,10.16,120582.0,-21897.0,41420.0,-0.69,-15.37,34.35,13.70,163.0,-0.53,0.88,42.32,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1166.35,63.0,M-SC,12.67,172280.0,-8863.0,54337.0,-0.95,-4.89,31.54,25.10,235.0,-0.16,1.26,38.07,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.88,82356.0,-19854.0,113248.0,0.62,-19.42,137.51,91.37,208.0,-0.18,0.60,33.69,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.00,53.72,51.0,H-SC,5.62,133217.0,-49558.0,160633.0,-2.00,-27.11,120.58,60.77,125.0,-0.31,0.97,19.32,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3591.89,49.0,M-SC,7.12,115085.0,-32283.0,84093.0,-0.82,-21.91,73.07,35.16,236.0,-0.38,0.84,17.55,XY24,NTT,AUTO
71,TANLA,1963.11,-19.51,56.0,H-SC,17.00,215715.0,-55046.0,385008.0,-2.18,-20.33,178.48,121.86,133.0,-0.14,1.58,68.75,AR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.77
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.86
LC    32.48
MC    23.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.33
X40      14.75
X40N     11.37
XR       11.15
XY25     10.18
AR        8.32
OX40N     7.90
X200      1.68
MH        1.62
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.35
H-LC    25.99
H-MC    20.67
M-SC    13.04
M-LC     5.44
M-MC     2.65
L-SC     1.47
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.75,41.98
IT,12.84,-19.71,84.99
FINANCE,10.06,-19.22,71.24
MISC,6.99,-20.10,86.87
BANKS,6.17,-18.39,80.67
PAINTS,5.66,-19.10,36.86
ELECTRICAL,5.47,-9.96,48.54
INSURANCE,3.78,-6.88,46.83
AC,3.57,0.95,15.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3232376.0,22
XR,1385932.0,14
AR,1193297.0,9
X40,832241.0,10
X40N,646286.0,9
OX40N,564493.0,10
XY25,446805.0,7
SR,254971.0,2
X5K,120961.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3775988.0,29
M-SC,1460850.0,17
H-MC,1278867.0,15
H-LC,1265103.0,15
M-LC,406236.0,4
M-MC,315337.0,2
L-SC,261752.0,3
L-MC,61560.0,1
L-LC,40225.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1254617.0      6
M-SC       XY24       836601.0      7
H-SC       AR         812150.0      5
           XR         802839.0      7
H-MC       XY24       577622.0      4
H-LC       X40        527654.0      5
H-SC       X40N       317967.0      4
M-MC       XY24       315337.0      2
H-SC       OX40N      263964.0      4
           SR         254971.0      2
H-LC       X40N       242155.0      3
H-MC       X40        227988.0      4
H-LC       AR         210482.0      2
H-MC       XY25       180830.0      2
L-SC       XR         173095.0      2
M-LC       XY24       171828.0      2
M-SC       AR         170665.0      2
           OX40N      139247.0      4
           XR         124490.0      2
M-LC       X5K        120961.0      1
           XR         113447.0      1
M-SC       XY25       113248.0      1
H-LC       XY25       112502.0      3
H-MC       XR         110501.0      1
H-LC       X200        95939.0      1
L-SC       OX40N       88657.0      1
H-MC       X40N        86164.0      2
M-SC       X40         76599.0      1
H-LC       XY24        76371.0      1
H-MC       OX40N       72625.0      1
H-SC       MH          69480.0      1
L-MC       XR          61560.0      1
L-LC       XY25        40225.0      1
H-MC       MH          23137.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
